 # 🔧 Semantic Kernel Tool Use Example – Research Assistant



 Welcome to this **Semantic Kernel Tool Use** sample! This code demonstrates how to build a powerful **Research Assistant** using the Semantic Kernel framework. You'll learn how to create an AI agent that can search for academic papers, perform web searches, generate summaries, and create visualizations.



 ## 🎯 **Objective**



 The goal of this sample is to demonstrate how to:

 - Create and register **custom plugins** as tools for your AI agent

 - Configure **function calling behavior** to control how your agent uses tools

 - Build a complete **research assistant** that can gather and analyze information

 - Implement an **interactive UI** for engaging with your AI agent



 ## 🧩 **Why Plugins Matter**



 Plugins in Semantic Kernel are essential because they:

 - Extend your AI agent's capabilities beyond just conversation

 - Allow your agent to interact with external systems and APIs

 - Enable complex workflows by combining multiple tools

 - Create a modular architecture that's easy to maintain and expand



 ## 🔄 **Function Calling Behavior**



 This sample demonstrates how to control when and how your agent uses tools through:

 - **Auto-invocation** of functions when the agent decides they're needed

 - **Required function choice** to ensure the agent uses available tools

 - **Maximum invocation attempts** to prevent infinite loops



 ## 💡 **Key Takeaways from this Sample**



 - Learn how to create custom plugins with multiple functions

 - Understand how to configure function calling behavior

 - See how to build a complete research workflow with multiple tools

 - Explore techniques for creating interactive AI agent interfaces



 Let's get started! 🚀

 ## 📦 Import the Needed Packages



 We'll start by importing all the necessary libraries for our research assistant. These include:



 - **Semantic Kernel core components** for building our agent

 - **External APIs** for searching academic papers and the web

 - **Data processing libraries** for handling and visualizing information

 - **UI components** for creating an interactive interface

In [1]:
# Standard library imports
import os
import asyncio
import json
import re
import random
from datetime import datetime
from typing import Annotated

# Third-party imports
import arxiv
import pandas as pd
import matplotlib.pyplot as plt
from duckduckgo_search import DDGS
import markdownify
from IPython.display import display, Markdown, clear_output
import ipywidgets as widgets

# Semantic Kernel imports
from semantic_kernel.functions import kernel_function
from semantic_kernel.kernel import Kernel, KernelArguments
from semantic_kernel.agents import ChatCompletionAgent
from semantic_kernel.contents import ChatHistory
from semantic_kernel.contents.function_call_content import FunctionCallContent
from semantic_kernel.contents.function_result_content import FunctionResultContent
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.ai.function_choice_behavior import (
    FunctionChoiceBehavior,
)



In [2]:
# Initialize constants
SERVICE_ID = "agent"
OUTPUT_DIRS = ["papers", "visualizations", "summaries"]


def create_directories():
    """Create directories for saving files."""
    for directory in OUTPUT_DIRS:
        os.makedirs(directory, exist_ok=True)

    print(f"Created directories: {', '.join(OUTPUT_DIRS)}")


# Initialize kernel and service
kernel = Kernel()
kernel.add_service(AzureChatCompletion(service_id=SERVICE_ID))
settings = kernel.get_prompt_execution_settings_from_service_id(service_id=SERVICE_ID)

# Create necessary directories
create_directories()


ServiceInitializationError: chat_deployment_name is required.

 ## 🏗️ **Setting Up the Environment**



 Before we start building our research assistant, we need to set up a directory structure to store:

 - Downloaded academic papers

 - Generated visualizations

 - Research summaries and web content



 The `create_directories()` function ensures these folders exist, creating them if necessary.



 This organization helps maintain a clean project structure and makes it easy to find and manage the files our research assistant will generate.

 ## 🔌 **Creating the Plugins**



 Semantic Kernel uses plugins as tools that can be called by the agent. A plugin can have multiple `kernel_functions` in it as a group.



 In this example, we'll create several plugins that work together to form a complete research workflow:

 1. **ArXivPlugin** - Searches for and downloads academic papers

 2. **WebSearchPlugin** - Performs web searches and extracts content from web pages

 3. **ResearchPlugin** - Generates research summaries from collected information

 4. **VisualizationPlugin** - Creates data visualizations to help understand research topics



 Each plugin contains one or more functions decorated with `@kernel_function`, which makes them available to the AI agent.



 ### 💡 **Why Use Multiple Plugins?**

 - **Separation of concerns** - Each plugin handles a specific aspect of the research process

 - **Modularity** - Plugins can be reused across different projects

 - **Maintainability** - Easier to update or extend individual components

 - **Clarity** - Makes the agent's capabilities more explicit and organized

In [ ]:
# Define the ArXiv Search Plugin with PDF download
class ArXivPlugin:
    """Plugin for searching academic papers on arXiv and downloading PDFs."""

    @kernel_function(
        description="Search for academic papers on arXiv based on a query."
    )
    def search_papers(
        self,
        query: Annotated[str, "The research topic to search for on arXiv."],
        max_results: Annotated[int, "Maximum number of results to return."] = 5,
    ) -> Annotated[str, "JSON string of arXiv paper results."]:
        """Search for papers on arXiv related to the query."""
        try:
            # Search arXiv
            search = arxiv.Search(
                query=query,
                max_results=max_results,
                sort_by=arxiv.SortCriterion.Relevance,
            )

            results = []
            for paper in search.results():
                # Format the authors and prepare filename
                authors = ", ".join([author.name for author in paper.authors])
                arxiv_id = paper.entry_id.split("/")[-1]

                # Create a clean filename from the title
                clean_title = re.sub(r"[^\w\s-]", "", paper.title)
                clean_title = re.sub(r"[-\s]+", "-", clean_title).strip("-")[:50]
                filename = f"{arxiv_id}_{clean_title}.pdf"
                filepath = os.path.join("papers", filename)

                # Download the PDF file
                try:
                    paper.download_pdf("papers")
                    download_status = "success"
                except Exception as e:
                    download_status = f"failed: {str(e)}"

                # Add paper information to results
                results.append(
                    {
                        "title": paper.title,
                        "authors": authors,
                        "summary": paper.summary.replace("\n", " "),
                        "published": paper.published.strftime("%Y-%m-%d"),
                        "url": paper.pdf_url,
                        "arxiv_id": arxiv_id,
                        "local_path": filepath
                        if download_status == "success"
                        else None,
                        "download_status": download_status,
                    }
                )

            if not results:
                return json.dumps(
                    {
                        "status": "no_results",
                        "message": "No papers found on arXiv for this query.",
                    }
                )

            return json.dumps({"status": "success", "papers": results}, indent=2)

        except Exception as e:
            return json.dumps({"status": "error", "message": str(e)})



 ### 🌐 **Web Search Plugin**



 The **WebSearchPlugin** extends our research capabilities beyond academic papers by:

 - Searching the web using DuckDuckGo's API

 - Extracting and processing content from web pages

 - Converting HTML content to markdown for easier processing

 - Saving web content to files for future reference



 This plugin is particularly useful when:

 - Academic papers don't provide enough information

 - We need more recent information than what's available in published papers

 - We want to include non-academic perspectives on a topic

In [ ]:
class WebSearchPlugin:
    """Plugin for web search using DuckDuckGo."""

    @kernel_function(description="Search the web using DuckDuckGo.")
    def search_web(
        self,
        query: Annotated[str, "The search query to look up."],
        max_results: Annotated[int, "Maximum number of results to return."] = 5,
    ) -> Annotated[str, "JSON string of web search results."]:
        """Perform a web search using DuckDuckGo for a given query."""
        try:
            ddgs = DDGS()
            results = list(ddgs.text(query, max_results=max_results))

            if not results:
                return json.dumps(
                    {
                        "status": "no_results",
                        "message": "No web results found for this query.",
                    }
                )

            return json.dumps({"status": "success", "results": results}, indent=2)

        except Exception as e:
            return json.dumps({"status": "error", "message": str(e)})

    @kernel_function(
        description="Get the content of a web page and convert it to markdown."
    )
    def get_page_content(
        self, url: Annotated[str, "The URL of the web page to extract content from."]
    ) -> Annotated[str, "Markdown content of the web page."]:
        """Extract content from a web page and convert it to markdown."""
        try:
            import requests
            from bs4 import BeautifulSoup

            # Set up request headers for a standard browser
            headers = {"User-Agent": "Mozilla/5.0"}

            # Get the web page content
            response = requests.get(url, headers=headers, timeout=10)
            response.raise_for_status()

            # Parse the HTML content
            soup = BeautifulSoup(response.text, "html.parser")

            # Remove script and style elements that aren't needed
            for element in soup(["script", "style"]):
                element.extract()

            # Try to find the main content using common HTML5 semantic elements
            # Fall back to body if no semantic elements are found
            main_content = (
                soup.find("main") or soup.find("article") or soup.find("body")
            )

            if not main_content:
                return "Could not extract meaningful content from the page."

            # Convert to markdown
            md = markdownify.markdownify(str(main_content), heading_style="ATX")

            # Generate a unique filename with timestamp
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"web_content_{timestamp}.md"
            filepath = os.path.join("summaries", filename)

            # Save the full content to a file
            with open(filepath, "w", encoding="utf-8") as f:
                f.write(f"# Content from {url}\n\n")
                f.write(md)

            # Truncate content for return value if it's too long
            # 25,000 chars is approximately 5000 words
            MAX_CONTENT_LENGTH = 25000
            if len(md) > MAX_CONTENT_LENGTH:
                md = (
                    md[:MAX_CONTENT_LENGTH]
                    + "\n\n... (content truncated due to length)"
                )

            return (
                f"Content extracted and saved to {filepath}. Preview:\n\n{md[:1000]}..."
            )

        except Exception as e:
            return f"Error retrieving or processing web page: {str(e)}"



 ### 📝 **Research Plugin**



 The **ResearchPlugin** helps synthesize information from multiple sources into a coherent summary. In a real-world implementation, this would likely use an LLM to:

 - Analyze content from academic papers and web searches

 - Identify key findings and methodologies

 - Generate a structured research summary

 - Save the summary to a file for future reference



 This plugin demonstrates how Semantic Kernel can be used to create higher-level cognitive functions that build on the data gathering capabilities of other plugins.

In [ ]:
class ResearchPlugin:
    """Plugin for analyzing research information and generating summaries."""

    @kernel_function(
        description="Generate a research summary from papers and web content."
    )
    def generate_summary(
        self,
        topic: Annotated[str, "The research topic being investigated."],
        papers_json: Annotated[str, "JSON string of papers from arXiv (may be empty)."],
        web_content: Annotated[str, "Content from web pages (may be empty)."],
    ) -> Annotated[str, "A comprehensive research summary."]:
        """Generate a research summary and save it to a file."""
        # Create a summary template (in a real implementation, this would call an LLM)
        summary_template = """
# Research Summary: {topic}

## Sources
The research summary is based on available papers from arXiv and web content related to the topic.

## Key Findings
- Finding 1 related to {topic}
- Finding 2 related to {topic}
- Finding 3 related to {topic}

## Methodology
Various research methodologies were observed across the literature...

## Current State
The current state of research on {topic}...

## Future Directions
Based on the analyzed content, future research might focus on...

## Conclusion
This research summary provides an overview of the current understanding of {topic}...
"""
        # Format the summary with the topic
        summary = summary_template.format(topic=topic)

        # Create a clean filename from the topic
        clean_topic = re.sub(r"[^\w\s-]", "", topic)
        clean_topic = re.sub(r"[-\s]+", "-", clean_topic).strip("-")

        # Generate a unique filename with timestamp
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"summary_{clean_topic}_{timestamp}.md"
        filepath = os.path.join("summaries", filename)

        with open(filepath, "w", encoding="utf-8") as f:
            f.write(summary)

        return f"Research summary created and saved to {filepath}.\n\n{summary}"



 ### 📊 **Visualization Plugin**



 The **VisualizationPlugin** adds data visualization capabilities to our research assistant. It can:

 - Generate sample data based on a research topic

 - Create different types of charts (bar, line, scatter)

 - Save visualizations to disk for inclusion in reports



 Visualizations are powerful tools for understanding complex information and identifying patterns. This plugin demonstrates how Semantic Kernel can integrate with data visualization libraries like matplotlib and pandas.

In [ ]:
class VisualizationPlugin:
    """Plugin for creating visualizations from research data."""

    @kernel_function(
        description="Generate sample data for visualization based on a research topic."
    )
    def generate_visualization_data(
        self, topic: Annotated[str, "The research topic being visualized."]
    ) -> Annotated[str, "JSON data for visualization."]:
        """Generate sample data for visualization (simulated for this example)."""
        # Extract words from the topic for category names
        topic_words = re.findall(r"\w+", topic.lower())

        # Number of categories to generate
        NUM_CATEGORIES = 5

        # Generate category names based on topic words
        categories = []
        for i in range(NUM_CATEGORIES):
            if topic_words:
                word = random.choice(topic_words)
                categories.append(f"{word.capitalize()} {i+1}")
            else:
                categories.append(f"Category {i+1}")

        # Generate random metrics for visualization
        impact_factor = [random.randint(10, 100) for _ in range(len(categories))]
        research_activity = [random.randint(20, 80) for _ in range(len(categories))]

        # Create the data structure for visualization
        data = {
            "categories": categories,
            "metrics": {
                "Impact Factor": impact_factor,
                "Research Activity": research_activity,
            },
        }

        return json.dumps(data, indent=2)

    @kernel_function(
        description="Create a visualization from provided data and save it to disk."
    )
    def create_visualization(
        self,
        data_json: Annotated[str, "JSON data for visualization."],
        chart_type: Annotated[
            str, "Type of chart to create (bar, line, scatter, etc.)."
        ] = "bar",
        topic: Annotated[
            str, "The research topic for the chart title."
        ] = "Research Topic",
    ) -> Annotated[str, "Path to the saved visualization."]:
        """Create a visualization from the provided data and save it to disk."""
        try:
            # Parse the JSON data
            data = json.loads(data_json)

            # Set the style
            plt.style.use("ggplot")

            # Create figure
            plt.figure(figsize=(10, 6))

            # Extract data
            categories = data["categories"]
            metrics = data["metrics"]

            # Create a clean filename from the topic
            clean_topic = re.sub(r"[^\w\s-]", "", topic)
            clean_topic = re.sub(r"[-\s]+", "-", clean_topic).strip("-")

            # Generate a timestamp for unique filenames
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

            # Determine chart type and create visualization
            if chart_type == "bar":
                # Set up the data for plotting
                df = pd.DataFrame({"Categories": categories, **metrics})
                df.set_index("Categories", inplace=True)

                # Create the bar chart with rotated x-axis labels
                ax = df.plot(kind="bar", rot=45)
                plt.title(f"{topic} - Research Metrics by Category")
                plt.ylabel("Value")
                plt.legend(title="Metrics")
                filename = f"bar_chart_{clean_topic}_{timestamp}.png"

            elif chart_type == "line":
                # Set up the data for plotting
                df = pd.DataFrame({"Categories": categories, **metrics})
                df.set_index("Categories", inplace=True)

                # Create the line chart with markers at data points
                ax = df.plot(kind="line", marker="o")
                plt.title(f"{topic} - Research Metrics Trend by Category")
                plt.ylabel("Value")
                plt.legend(title="Metrics")
                filename = f"line_chart_{clean_topic}_{timestamp}.png"

            elif chart_type == "scatter":
                # For scatter, we need exactly two metrics
                if len(metrics) >= 2:
                    metric_names = list(metrics.keys())
                    x_values = metrics[metric_names[0]]
                    y_values = metrics[metric_names[1]]

                    plt.scatter(x_values, y_values)
                    plt.xlabel(metric_names[0])
                    plt.ylabel(metric_names[1])
                    plt.title(
                        f"{topic} - Correlation between {metric_names[0]} and {metric_names[1]}"
                    )

                    # Add category labels to each point in the scatter plot
                    for i, category in enumerate(categories):
                        plt.annotate(
                            category,  # Text label
                            (x_values[i], y_values[i]),  # Point position
                            textcoords="offset points",  # How to position text
                            xytext=(0, 10),  # Text offset
                            ha="center",  # Horizontal alignment
                        )

                    filename = f"scatter_chart_{clean_topic}_{timestamp}.png"
                else:
                    return "Scatter plot requires at least two metrics."
            else:
                return f"Unsupported chart type: {chart_type}"

            # Save the chart to disk
            filepath = os.path.join("visualizations", filename)
            plt.tight_layout()
            plt.savefig(filepath, dpi=300)
            plt.close()

            return f"Visualization created and saved to {filepath}"

        except Exception as e:
            return f"Error creating visualization: {str(e)}"



 ## 🤖 **Creating a Conversation Helper**



 The `continue_research_conversation` function helps manage the conversation flow between the user and the research assistant. It:

 - Adds the user's message to the chat history

 - Streams the agent's response, filtering out function calls and results

 - Returns the updated chat history for future interactions



 This function demonstrates how to handle streaming responses from Semantic Kernel agents, which is important for providing a responsive user experience, especially when the agent is performing complex operations like searching for papers or generating visualizations.

In [ ]:
async def continue_research_conversation(agent, chat_history, user_message):
    """
    Process a user message and stream the agent's response.

    Args:
        agent: The ChatCompletionAgent to use for generating responses
        chat_history: The conversation history to add to
        user_message: The new message from the user

    Returns:
        The updated chat history
    """
    # Add the user's message to the chat history
    chat_history.add_user_message(user_message)
    print(f"# User: '{user_message}'")

    # Start streaming the agent's response
    print("# ResearchAssistant: '", end="")

    # Process the response stream
    async for content in agent.invoke_stream(chat_history):
        # Only print non-function content (skip function calls and results)
        is_function_content = any(
            isinstance(item, (FunctionCallContent, FunctionResultContent))
            for item in content.items
        )

        if not is_function_content and content.content.strip():
            print(f"{content.content}", end="", flush=True)

    print("'")
    return chat_history



 ## 🧠 **Configuring the Research Assistant**



 Now we'll set up our research assistant by:

 1. Defining the agent's name and instructions

 2. Configuring the kernel and AI service

 3. Setting up function choice behavior

 4. Registering our plugins

 5. Creating the agent with the configured settings



 The agent instructions are particularly important as they define:

 - The agent's role and capabilities

 - The workflow it should follow

 - How it should use its available tools

 - Where it should save different types of files

## 🔄 **Setting the Function Choice Behavior**



 In Semantic Kernel, we have the ability to control the agent's choice of functions. This is done using the `FunctionChoiceBehavior` class.



 The code in this example sets it to `Required(auto_invoke=True)` which:

 - **Requires** the agent to choose at least one function when appropriate

 - **Auto-invokes** the chosen functions without additional confirmation

 - Allows up to 100 function invocations in a single conversation turn



 This can also be set to:

 - `FunctionChoiceBehavior.Auto()` - allows the agent to choose among the available functions or not choose any

 - `FunctionChoiceBehavior.NoneInvoke()` - instructs the agent to not choose any function (good for testing)

In [ ]:
# Define the agent instructions
AGENT_NAME = "ResearchAssistant"
AGENT_INSTRUCTIONS = """
You are a helpful research assistant that can search for information on academic and general topics.

Your capabilities include:
- Searching for academic papers on arXiv
- Searching the web using DuckDuckGo
- Extracting and processing content from web pages
- Generating research summaries
- Creating data visualizations

When given a research topic:
1. First try to find relevant academic papers on arXiv
2. If no relevant papers are found, use web search as a fallback
3. Process and analyze the content you find
4. Generate a summary of your findings
5. Create visualizations when appropriate

You should decide on your own which tools to use and in what order based on the user's needs.
Always explain what you're doing and why.

Important:
- You can download PDF papers from ArXiv to the 'papers' directory
- You should save visualizations to the 'visualizations' directory
- Research summaries and save to the 'summaries' directory
"""

# Configure function calling behavior
settings.function_choice_behavior = FunctionChoiceBehavior.Required(auto_invoke=True)
settings.function_choice_behavior.enable_kernel_functions = True
settings.function_choice_behavior.maximum_auto_invoke_attempts = 10

# Register all plugins
kernel.add_plugin(ArXivPlugin(), plugin_name="arxiv")
kernel.add_plugin(WebSearchPlugin(), plugin_name="web")
kernel.add_plugin(ResearchPlugin(), plugin_name="research")
kernel.add_plugin(VisualizationPlugin(), plugin_name="visualization")

# Create the research agent
agent = ChatCompletionAgent(
    service_id=SERVICE_ID,
    kernel=kernel,
    name=AGENT_NAME,
    instructions=AGENT_INSTRUCTIONS,
    arguments=KernelArguments(settings=settings),
)

# Initialize chat history
chat = ChatHistory()



 ## 🔍 **Testing the Research Assistant**



 Let's create a helper function to test our research assistant with a simple query. This function:

 - Adds the user's question to the chat history

 - Prints the user's question

 - Streams the agent's response in real-time

 - Adds a newline at the end for readability



 We'll use this function to test our research assistant with a query about quantum computing.

In [ ]:
async def ask_research(question: str):
    """
    Helper function to send a research query and stream the agent's response.

    Args:
        question: The research question or topic to investigate
    """
    # Add the question to chat history
    chat.add_user_message(question)
    print(f"User: {question}")

    # Stream the response
    async for response in agent.invoke_stream(chat):
        print(response.content, end="", flush=True)

    print("\n")


# Example usage with research queries
# To run this in a Jupyter notebook cell:
await ask_research("Research quantum computing")

# To run this in a regular Python script:
# asyncio.run(ask_research("Research quantum computing"))


 ## 🖥️ **Building an Interactive UI**



 To make our research assistant more user-friendly, we'll create an interactive UI using IPython widgets. This UI includes:

 - A text input for entering research topics

 - A button to start the research process

 - A text input for follow-up questions

 - A status indicator to show the current state

 - An output area to display the conversation



 This demonstrates how Semantic Kernel agents can be integrated into interactive applications, providing a more engaging user experience.

In [ ]:
class ResearchAssistantApp:
    """Interactive UI for the Research Assistant using IPython widgets."""

    def __init__(self):
        self.agent = None
        self.chat_history = None
        self.kernel = None
        self.service_id = None  # Store the service_id
        self.create_ui()

    def create_ui(self):
        """Create and display the UI components."""
        # Define UI component dimensions
        input_width = "70%"
        button_width = "20%"

        # Create the query input field
        self.query_input = widgets.Text(
            value="",
            placeholder="Enter a research topic...",
            description="Query:",
            layout=widgets.Layout(width=input_width),
        )

        # Create the search button
        self.search_button = widgets.Button(
            description="Research",
            button_style="primary",
            layout=widgets.Layout(width=button_width),
        )
        self.search_button.on_click(self.on_search_clicked)

        # Create the input box (horizontal container)
        self.input_box = widgets.HBox([self.query_input, self.search_button])

        # Create the follow-up message input (initially disabled)
        self.message_input = widgets.Text(
            value="",
            placeholder="Ask a follow-up question...",
            description="Message:",
            layout=widgets.Layout(width=input_width),
            disabled=True,
        )

        # Create the send button (initially disabled)
        self.send_button = widgets.Button(
            description="Send",
            button_style="info",
            layout=widgets.Layout(width=button_width),
            disabled=True,
        )
        self.send_button.on_click(self.on_send_clicked)

        # Create the message box (horizontal container)
        self.message_box = widgets.HBox([self.message_input, self.send_button])

        # Create the output display area
        self.output = widgets.Output(
            layout=widgets.Layout(
                border="1px solid #ddd",
                padding="10px",
                height="500px",
                overflow_y="auto",
            )
        )

        # Create the status indicator
        self.status = widgets.HTML(value="<i>Ready to start research.</i>")

        # Display all UI components
        display(self.input_box)
        display(self.message_box)
        display(self.status)
        display(self.output)

    async def initialize(self):
        """Initialize the kernel, plugins, and agent."""
        self.status.value = "<i>Initializing kernel and plugins...</i>"

        # Create a new kernel
        self.kernel = Kernel()
        self.service_id = "agent"

        # Register the AI service
        self.kernel.add_service(AzureChatCompletion(service_id=self.service_id))

        # Configure function calling behavior
        settings = self.kernel.get_prompt_execution_settings_from_service_id(
            service_id=self.service_id
        )
        settings.function_choice_behavior = FunctionChoiceBehavior.Required(
            auto_invoke=True
        )
        settings.function_choice_behavior.enable_kernel_functions = True
        settings.function_choice_behavior.maximum_auto_invoke_attempts = 100

        # Register all plugins
        self.kernel.add_plugin(ArXivPlugin(), plugin_name="arxiv")
        self.kernel.add_plugin(WebSearchPlugin(), plugin_name="web")
        self.kernel.add_plugin(ResearchPlugin(), plugin_name="research")
        self.kernel.add_plugin(VisualizationPlugin(), plugin_name="visualization")

        # Create the agent with the configured settings
        self.agent = ChatCompletionAgent(
            service_id=self.service_id,
            kernel=self.kernel,
            name=AGENT_NAME,
            instructions=AGENT_INSTRUCTIONS,
            arguments=KernelArguments(settings=settings),
        )

        # Initialize chat history
        self.chat_history = ChatHistory()

        # Update status
        self.status.value = "<i>Ready!</i>"

    def on_search_clicked(self, button):
        """Handle search button clicks."""
        # Get the query text
        query = self.query_input.value

        # Validate input
        if not query:
            self.status.value = (
                "<b style='color:red'>Please enter a research topic!</b>"
            )
            return

        # Disable inputs during processing
        self.query_input.disabled = True
        self.search_button.disabled = True

        # Update status
        self.status.value = "<i>Researching...</i>"

        # Clear previous output
        with self.output:
            clear_output()

        # Start asynchronous processing
        asyncio.create_task(self.process_query(query))

    async def process_query(self, query):
        # Initialize if needed
        if self.kernel is None:
            await self.initialize()

        # Create new chat history
        self.chat_history = ChatHistory()

        # Add the query to chat history
        self.chat_history.add_user_message(f"Research this topic: {query}")

        # Display user message
        with self.output:
            display(Markdown(f"**User**: Research this topic: {query}"))

        # Get agent response
        with self.output:
            display(Markdown("**ResearchAssistant**: _Thinking..._"))

        # Collect response parts
        response_parts = []

        # Stream the response
        async for content in self.agent.invoke_stream(self.chat_history):
            # Collect non-function content
            if (
                not any(
                    isinstance(item, (FunctionCallContent, FunctionResultContent))
                    for item in content.items
                )
                and content.content.strip()
            ):
                response_parts.append(content.content)
                # Update the output with the current response
                with self.output:
                    clear_output(wait=True)
                    display(Markdown(f"**User**: Research this topic: {query}"))
                    display(
                        Markdown(f"**ResearchAssistant**: {''.join(response_parts)}")
                    )

        # Enable follow-up messages
        self.message_input.disabled = False
        self.send_button.disabled = False
        self.status.value = "<i>Research complete. You can ask follow-up questions.</i>"

    def on_send_clicked(self, button):
        message = self.message_input.value
        if not message:
            self.status.value = "<b style='color:red'>Please enter a message!</b>"
            return

        # Disable input while processing
        self.message_input.disabled = True
        self.send_button.disabled = True
        self.status.value = "<i>Processing...</i>"

        # Process message asynchronously
        asyncio.create_task(self.process_message(message))

        # Clear the input
        self.message_input.value = ""

    async def process_message(self, message):
        # Add the message to chat history
        self.chat_history.add_user_message(message)

        # Display user message
        with self.output:
            display(Markdown(f"**User**: {message}"))

        # Collect response parts
        response_parts = []

        # Stream the response
        async for content in self.agent.invoke_stream(self.chat_history):
            # Collect non-function content
            if (
                not any(
                    isinstance(item, (FunctionCallContent, FunctionResultContent))
                    for item in content.items
                )
                and content.content.strip()
            ):
                response_parts.append(content.content)
                # Update the output
                with self.output:
                    display(
                        Markdown(f"**ResearchAssistant**: {''.join(response_parts)}")
                    )

        # Re-enable input
        self.message_input.disabled = False
        self.send_button.disabled = False
        self.status.value = "<i>Ready for more questions.</i>"

app = ResearchAssistantApp()